# Combined Feature extracted from RNN and LSTM(64 colonms/feature for each image)


In [42]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.utils import to_categorical

In [43]:
# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
input_dim = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape((-1, 1, input_dim))
X_test = X_test.reshape((-1, 1, input_dim))
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [44]:
# One-hot encode the target labels
num_classes = 10
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

In [4]:
# Build and train the RNN model
model = Sequential()
model.add(SimpleRNN(64, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [5]:
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 11s - loss: 0.2992 - accuracy: 0.9147 - 11s/epoch - 6ms/step
Epoch 2/10
1875/1875 - 9s - loss: 0.1456 - accuracy: 0.9577 - 9s/epoch - 5ms/step
Epoch 3/10
1875/1875 - 8s - loss: 0.1056 - accuracy: 0.9688 - 8s/epoch - 4ms/step
Epoch 4/10
1875/1875 - 10s - loss: 0.0834 - accuracy: 0.9752 - 10s/epoch - 5ms/step
Epoch 5/10
1875/1875 - 10s - loss: 0.0676 - accuracy: 0.9800 - 10s/epoch - 6ms/step
Epoch 6/10
1875/1875 - 9s - loss: 0.0575 - accuracy: 0.9827 - 9s/epoch - 5ms/step
Epoch 7/10
1875/1875 - 10s - loss: 0.0485 - accuracy: 0.9854 - 10s/epoch - 5ms/step
Epoch 8/10
1875/1875 - 10s - loss: 0.0407 - accuracy: 0.9877 - 10s/epoch - 5ms/step
Epoch 9/10
1875/1875 - 10s - loss: 0.0358 - accuracy: 0.9891 - 10s/epoch - 5ms/step
Epoch 10/10
1875/1875 - 10s - loss: 0.0298 - accuracy: 0.9910 - 10s/epoch - 5ms/step


In [6]:
# Extract features from the trained RNN model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [7]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))

In [8]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [9]:
# Save features and labels to Excel files
train_df.to_excel('rnn_train_features_labels.xlsx', index=False)
test_df.to_excel('rnn_test_features_labels.xlsx', index=False)

# 64 Features from MNIST using LSTM

In [10]:
from keras.layers import LSTM, Dense
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess the data
input_dim = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape((-1, 1, input_dim))
X_test = X_test.reshape((-1, 1, input_dim))
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [11]:
num_classes = 10
memory_cell = 64

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(memory_cell, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [12]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 13s - loss: 0.2720 - accuracy: 0.9240 - 13s/epoch - 7ms/step
Epoch 2/10
1875/1875 - 13s - loss: 0.1150 - accuracy: 0.9660 - 13s/epoch - 7ms/step
Epoch 3/10
1875/1875 - 14s - loss: 0.0766 - accuracy: 0.9775 - 14s/epoch - 8ms/step
Epoch 4/10
1875/1875 - 12s - loss: 0.0568 - accuracy: 0.9830 - 12s/epoch - 6ms/step
Epoch 5/10
1875/1875 - 13s - loss: 0.0429 - accuracy: 0.9867 - 13s/epoch - 7ms/step
Epoch 6/10
1875/1875 - 13s - loss: 0.0338 - accuracy: 0.9897 - 13s/epoch - 7ms/step
Epoch 7/10
1875/1875 - 13s - loss: 0.0259 - accuracy: 0.9918 - 13s/epoch - 7ms/step
Epoch 8/10
1875/1875 - 12s - loss: 0.0213 - accuracy: 0.9932 - 12s/epoch - 6ms/step
Epoch 9/10
1875/1875 - 13s - loss: 0.0171 - accuracy: 0.9947 - 13s/epoch - 7ms/step
Epoch 10/10
1875/1875 - 13s - loss: 0.0157 - accuracy: 0.9947 - 13s/epoch - 7ms/step


In [13]:
# Extract features and labels from the trained LSTM model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [14]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))


In [15]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [16]:
# Save features and labels to Excel files
train_df.to_excel('lstm_train_features_labels.xlsx', index=False)
test_df.to_excel('lstm_test_features_labels.xlsx', index=False)

# Save the combined features and labels to Excel files

In [23]:
import pandas as pd
from sklearn.utils import shuffle

# Load the extracted features and labels from both models
lstm_train_df = pd.read_excel('lstm_train_features_labels.xlsx')
lstm_test_df = pd.read_excel('lstm_test_features_labels.xlsx')
rnn_train_df = pd.read_excel('rnn_train_features_labels.xlsx')
rnn_test_df = pd.read_excel('rnn_test_features_labels.xlsx')

# Combine features column-wise
combined_train_df = pd.concat([lstm_train_df.drop('label', axis=1), rnn_train_df.drop('label', axis=1), lstm_train_df['label']], axis=1)
combined_test_df = pd.concat([lstm_test_df.drop('label', axis=1), rnn_test_df.drop('label', axis=1), lstm_test_df['label']], axis=1)

# Shuffle the combined data
combined_train_df = shuffle(combined_train_df)
combined_test_df = shuffle(combined_test_df)

# Save the combined features and labels to Excel files
combined_train_df.to_excel('combined_train_features_labels.xlsx', index=False)
combined_test_df.to_excel('combined_test_features_labels.xlsx', index=False)


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

# Apply k-NN with k=27

In [26]:
# Apply k-NN with k=27
knn_classifier = KNeighborsClassifier(n_neighbors=27)
knn_classifier.fit(X_train, y_train)

# Make predictions
y_pred = knn_classifier.predict(X_test)

In [27]:
# Evaluate performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [33]:

# Display the results
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.9734
F1 Score: 0.9734
Precision: 0.9735
Recall: 0.9734
Confusion Matrix:
[[ 970    0    1    0    0    1    4    1    1    2]
 [   0 1129    2    1    0    0    2    0    1    0]
 [  13    1  990    3    2    1    4    7   11    0]
 [   0    0    7  967    0    9    0   11   11    5]
 [   2    1    2    0  949    0    8    1    0   19]
 [   3    1    0    4    3  862   11    2    2    4]
 [   4    5    0    0    1    2  946    0    0    0]
 [   1   13    7    0    1    1    1  993    0   11]
 [   3    1    1    1    5    3    3    4  949    4]
 [   2    4    0    6    5    2    0    8    3  979]]


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

# Apply SVM with linear kernel

In [35]:
# Apply SVM with linear kernel
svm_linear_classifier = SVC(kernel='linear')
svm_linear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_linear = svm_linear_classifier.predict(X_test)

# Evaluate performance metrics for linear kernel
accuracy_linear = accuracy_score(y_test, y_pred_linear)
f1_linear = f1_score(y_test, y_pred_linear, average='weighted')
precision_linear = precision_score(y_test, y_pred_linear, average='weighted')
recall_linear = recall_score(y_test, y_pred_linear, average='weighted')
conf_matrix_linear = confusion_matrix(y_test, y_pred_linear)

In [38]:
# Display the results for linear kernel
print('Linear Kernel Results:')
print(f'Accuracy: {accuracy_linear:.4f}')
print(f'F1 Score: {f1_linear:.4f}')
print(f'Precision: {precision_linear:.4f}')
print(f'Recall: {recall_linear:.4f}')
print(f'ROC AUC: {roc_auc_linear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_linear)

Linear Kernel Results:
Accuracy: 0.9820
F1 Score: 0.9820
Precision: 0.9820
Recall: 0.9820
ROC AUC: 0.9899
Confusion Matrix:
[[ 970    0    2    0    1    0    3    1    2    1]
 [   0 1128    1    2    0    1    1    1    1    0]
 [   6    2 1009    2    1    0    0    6    6    0]
 [   0    0    1  995    0    5    0    3    3    3]
 [   0    1    6    0  962    0    3    2    1    7]
 [   2    0    0    8    1  873    3    1    1    3]
 [   3    2    0    1    2    4  946    0    0    0]
 [   0    3    8    3    1    0    0 1003    2    8]
 [   0    0    2    5    4    1    0    4  956    2]
 [   2    2    0    7   10    2    0    6    2  978]]


# Apply SVM with non-linear kernel

In [39]:
# Apply SVM with non-linear kernel 
svm_nonlinear_classifier = SVC(kernel='rbf') 
svm_nonlinear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_nonlinear = svm_nonlinear_classifier.predict(X_test)

# Evaluate performance metrics for non-linear kernel
accuracy_nonlinear = accuracy_score(y_test, y_pred_nonlinear)
f1_nonlinear = f1_score(y_test, y_pred_nonlinear, average='weighted')
precision_nonlinear = precision_score(y_test, y_pred_nonlinear, average='weighted')
recall_nonlinear = recall_score(y_test, y_pred_nonlinear, average='weighted')
conf_matrix_nonlinear = confusion_matrix(y_test, y_pred_nonlinear)


In [41]:
# Display the results for non-linear kernel
print('Non-Linear Kernel Results:')
print(f'Accuracy: {accuracy_nonlinear:.4f}')
print(f'F1 Score: {f1_nonlinear:.4f}')
print(f'Precision: {precision_nonlinear:.4f}')
print(f'Recall: {recall_nonlinear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_nonlinear)

Non-Linear Kernel Results:
Accuracy: 0.9845
F1 Score: 0.9845
Precision: 0.9845
Recall: 0.9845
Confusion Matrix:
[[ 971    0    2    0    1    1    2    1    1    1]
 [   0 1127    2    2    0    1    1    1    1    0]
 [   6    1 1015    1    1    0    1    3    3    1]
 [   0    0    1  996    0    4    0    5    3    1]
 [   1    0    5    0  962    0    3    1    0   10]
 [   2    0    0    4    1  877    2    1    3    2]
 [   2    2    0    1    2    4  947    0    0    0]
 [   0    3    7    1    1    0    0 1008    3    5]
 [   1    0    1    3    3    3    0    3  958    2]
 [   2    2    0    5    6    2    0    7    1  984]]


# 128 Features from MNIST using RNN

In [45]:
# Build and train the RNN model
model = Sequential()
model.add(SimpleRNN(128, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [46]:
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 8s - loss: 0.2531 - accuracy: 0.9292 - 8s/epoch - 4ms/step
Epoch 2/10
1875/1875 - 8s - loss: 0.1128 - accuracy: 0.9670 - 8s/epoch - 4ms/step
Epoch 3/10
1875/1875 - 7s - loss: 0.0793 - accuracy: 0.9763 - 7s/epoch - 4ms/step
Epoch 4/10
1875/1875 - 7s - loss: 0.0582 - accuracy: 0.9822 - 7s/epoch - 4ms/step
Epoch 5/10
1875/1875 - 7s - loss: 0.0451 - accuracy: 0.9864 - 7s/epoch - 4ms/step
Epoch 6/10
1875/1875 - 7s - loss: 0.0359 - accuracy: 0.9890 - 7s/epoch - 4ms/step
Epoch 7/10
1875/1875 - 7s - loss: 0.0285 - accuracy: 0.9910 - 7s/epoch - 4ms/step
Epoch 8/10
1875/1875 - 7s - loss: 0.0225 - accuracy: 0.9934 - 7s/epoch - 4ms/step
Epoch 9/10
1875/1875 - 7s - loss: 0.0189 - accuracy: 0.9941 - 7s/epoch - 4ms/step
Epoch 10/10
1875/1875 - 7s - loss: 0.0146 - accuracy: 0.9954 - 7s/epoch - 4ms/step


In [47]:
# Extract features from the trained RNN model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [48]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))

In [49]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [50]:
# Save features and labels to Excel files
train_df.to_excel('rnn_128_train_features_labels.xlsx', index=False)
test_df.to_excel('rnn_128_test_features_labels.xlsx', index=False)

# 128 Features from MNIST using LSTM

In [64]:
num_classes = 10
memory_cell = 128

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(memory_cell, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [65]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 14s - loss: 0.2514 - accuracy: 0.9283 - 14s/epoch - 7ms/step
Epoch 2/10
1875/1875 - 12s - loss: 0.1038 - accuracy: 0.9697 - 12s/epoch - 7ms/step
Epoch 3/10
1875/1875 - 13s - loss: 0.0683 - accuracy: 0.9800 - 13s/epoch - 7ms/step
Epoch 4/10
1875/1875 - 12s - loss: 0.0488 - accuracy: 0.9855 - 12s/epoch - 6ms/step
Epoch 5/10
1875/1875 - 12s - loss: 0.0341 - accuracy: 0.9901 - 12s/epoch - 7ms/step
Epoch 6/10
1875/1875 - 12s - loss: 0.0259 - accuracy: 0.9917 - 12s/epoch - 6ms/step
Epoch 7/10
1875/1875 - 12s - loss: 0.0209 - accuracy: 0.9937 - 12s/epoch - 7ms/step
Epoch 8/10
1875/1875 - 13s - loss: 0.0151 - accuracy: 0.9954 - 13s/epoch - 7ms/step
Epoch 9/10
1875/1875 - 15s - loss: 0.0130 - accuracy: 0.9961 - 15s/epoch - 8ms/step
Epoch 10/10
1875/1875 - 12s - loss: 0.0119 - accuracy: 0.9962 - 12s/epoch - 6ms/step


In [66]:
# Extract features and labels from the trained LSTM model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [67]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))


In [68]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [69]:
# Save features and labels to Excel files
train_df.to_excel('lstm_128_train_features_labels.xlsx', index=False)
test_df.to_excel('lstm_128_test_features_labels.xlsx', index=False)

# 256 Features from MNIST using RNN

In [51]:
# Build and train the RNN model
model = Sequential()
model.add(SimpleRNN(256, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [52]:
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 9s - loss: 0.2262 - accuracy: 0.9338 - 9s/epoch - 5ms/step
Epoch 2/10
1875/1875 - 8s - loss: 0.0937 - accuracy: 0.9721 - 8s/epoch - 4ms/step
Epoch 3/10
1875/1875 - 10s - loss: 0.0617 - accuracy: 0.9808 - 10s/epoch - 6ms/step
Epoch 4/10
1875/1875 - 8s - loss: 0.0444 - accuracy: 0.9860 - 8s/epoch - 5ms/step
Epoch 5/10
1875/1875 - 8s - loss: 0.0335 - accuracy: 0.9894 - 8s/epoch - 5ms/step
Epoch 6/10
1875/1875 - 8s - loss: 0.0244 - accuracy: 0.9927 - 8s/epoch - 4ms/step
Epoch 7/10
1875/1875 - 9s - loss: 0.0209 - accuracy: 0.9931 - 9s/epoch - 5ms/step
Epoch 8/10
1875/1875 - 9s - loss: 0.0147 - accuracy: 0.9952 - 9s/epoch - 5ms/step
Epoch 9/10
1875/1875 - 9s - loss: 0.0129 - accuracy: 0.9959 - 9s/epoch - 5ms/step
Epoch 10/10
1875/1875 - 9s - loss: 0.0132 - accuracy: 0.9957 - 9s/epoch - 5ms/step


In [53]:
# Extract features from the trained RNN model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [54]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))

In [55]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [56]:
# Save features and labels to Excel files
train_df.to_excel('rnn_256_train_features_labels.xlsx', index=False)
test_df.to_excel('rnn_256_test_features_labels.xlsx', index=False)

# 256 Features from MNIST using LSTM

In [76]:
num_classes = 10
memory_cell = 256

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(memory_cell, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [77]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 19s - loss: 0.2317 - accuracy: 0.9343 - 19s/epoch - 10ms/step
Epoch 2/10
1875/1875 - 15s - loss: 0.0911 - accuracy: 0.9727 - 15s/epoch - 8ms/step
Epoch 3/10
1875/1875 - 15s - loss: 0.0580 - accuracy: 0.9817 - 15s/epoch - 8ms/step
Epoch 4/10
1875/1875 - 14s - loss: 0.0384 - accuracy: 0.9877 - 14s/epoch - 8ms/step
Epoch 5/10
1875/1875 - 14s - loss: 0.0262 - accuracy: 0.9912 - 14s/epoch - 8ms/step
Epoch 6/10
1875/1875 - 15s - loss: 0.0195 - accuracy: 0.9940 - 15s/epoch - 8ms/step
Epoch 7/10
1875/1875 - 15s - loss: 0.0164 - accuracy: 0.9945 - 15s/epoch - 8ms/step
Epoch 8/10
1875/1875 - 15s - loss: 0.0128 - accuracy: 0.9956 - 15s/epoch - 8ms/step
Epoch 9/10
1875/1875 - 14s - loss: 0.0105 - accuracy: 0.9966 - 14s/epoch - 8ms/step
Epoch 10/10
1875/1875 - 15s - loss: 0.0106 - accuracy: 0.9967 - 15s/epoch - 8ms/step


In [78]:
# Extract features and labels from the trained LSTM model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [79]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))

In [80]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [81]:
# Save features and labels to Excel files
train_df.to_excel('lstm_256_train_features_labels.xlsx', index=False)
test_df.to_excel('lstm_256_test_features_labels.xlsx', index=False)

# 512 Features from MNIST using LSTM

In [82]:
num_classes = 10
memory_cell = 512

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(memory_cell, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [83]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 28s - loss: 0.2119 - accuracy: 0.9396 - 28s/epoch - 15ms/step
Epoch 2/10
1875/1875 - 27s - loss: 0.0793 - accuracy: 0.9760 - 27s/epoch - 14ms/step
Epoch 3/10
1875/1875 - 26s - loss: 0.0492 - accuracy: 0.9851 - 26s/epoch - 14ms/step
Epoch 4/10
1875/1875 - 26s - loss: 0.0338 - accuracy: 0.9893 - 26s/epoch - 14ms/step
Epoch 5/10
1875/1875 - 26s - loss: 0.0248 - accuracy: 0.9919 - 26s/epoch - 14ms/step
Epoch 6/10
1875/1875 - 26s - loss: 0.0170 - accuracy: 0.9945 - 26s/epoch - 14ms/step
Epoch 7/10
1875/1875 - 26s - loss: 0.0138 - accuracy: 0.9955 - 26s/epoch - 14ms/step
Epoch 8/10
1875/1875 - 26s - loss: 0.0117 - accuracy: 0.9963 - 26s/epoch - 14ms/step
Epoch 9/10
1875/1875 - 26s - loss: 0.0110 - accuracy: 0.9964 - 26s/epoch - 14ms/step
Epoch 10/10
1875/1875 - 26s - loss: 0.0091 - accuracy: 0.9970 - 26s/epoch - 14ms/step


In [84]:
# Extract features and labels from the trained LSTM model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [85]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))

In [86]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [87]:
# Save features and labels to Excel files
train_df.to_excel('lstm_512_train_features_labels.xlsx', index=False)
test_df.to_excel('lstm_512_test_features_labels.xlsx', index=False)

# 512 Features from MNIST using RNN


In [70]:
# Build and train the RNN model
model = Sequential()
model.add(SimpleRNN(512, input_shape=(1, input_dim), activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [71]:
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
1875/1875 - 12s - loss: 0.2006 - accuracy: 0.9411 - 12s/epoch - 6ms/step
Epoch 2/10
1875/1875 - 11s - loss: 0.0808 - accuracy: 0.9753 - 11s/epoch - 6ms/step
Epoch 3/10
1875/1875 - 11s - loss: 0.0544 - accuracy: 0.9828 - 11s/epoch - 6ms/step
Epoch 4/10
1875/1875 - 11s - loss: 0.0391 - accuracy: 0.9870 - 11s/epoch - 6ms/step
Epoch 5/10
1875/1875 - 12s - loss: 0.0264 - accuracy: 0.9920 - 12s/epoch - 6ms/step
Epoch 6/10
1875/1875 - 11s - loss: 0.0212 - accuracy: 0.9933 - 11s/epoch - 6ms/step
Epoch 7/10
1875/1875 - 12s - loss: 0.0170 - accuracy: 0.9941 - 12s/epoch - 6ms/step
Epoch 8/10
1875/1875 - 12s - loss: 0.0153 - accuracy: 0.9951 - 12s/epoch - 7ms/step
Epoch 9/10
1875/1875 - 11s - loss: 0.0128 - accuracy: 0.9954 - 11s/epoch - 6ms/step
Epoch 10/10
1875/1875 - 11s - loss: 0.0100 - accuracy: 0.9969 - 11s/epoch - 6ms/step


In [72]:
# Extract features from the trained RNN model
feature_extractor = Sequential(model.layers[:-1])  # Remove the last Dense layer
train_features = feature_extractor.predict(X_train)
test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [73]:
# Reshape features for saving to Excel
train_features = train_features.reshape((train_features.shape[0], -1))
test_features = test_features.reshape((test_features.shape[0], -1))

In [74]:
# Combine features with labels
train_df = pd.DataFrame(np.column_stack((train_features, y_train)), columns=[f'feature_{i}' for i in range(train_features.shape[1])] + ['label'])
test_df = pd.DataFrame(np.column_stack((test_features, y_test)), columns=[f'feature_{i}' for i in range(test_features.shape[1])] + ['label'])


In [75]:
# Save features and labels to Excel files
train_df.to_excel('rnn_512_train_features_labels.xlsx', index=False)
test_df.to_excel('rnn_512_test_features_labels.xlsx', index=False)

# Combined 128 Feature extracted from RNN and LSTM and Train KNN

In [88]:
import pandas as pd
from sklearn.utils import shuffle

# Load the extracted features and labels from both models
lstm_train_df = pd.read_excel('lstm_128_train_features_labels.xlsx')
lstm_test_df = pd.read_excel('lstm_128_test_features_labels.xlsx')
rnn_train_df = pd.read_excel('rnn_128_train_features_labels.xlsx')
rnn_test_df = pd.read_excel('rnn_128_test_features_labels.xlsx')

# Combine features column-wise
combined_train_df = pd.concat([lstm_train_df.drop('label', axis=1), rnn_train_df.drop('label', axis=1), lstm_train_df['label']], axis=1)
combined_test_df = pd.concat([lstm_test_df.drop('label', axis=1), rnn_test_df.drop('label', axis=1), lstm_test_df['label']], axis=1)

# Shuffle the combined data
combined_train_df = shuffle(combined_train_df)
combined_test_df = shuffle(combined_test_df)

# Save the combined features and labels to Excel files
combined_train_df.to_excel('combined_128_train_features_labels.xlsx', index=False)
combined_test_df.to_excel('combined_128_test_features_labels.xlsx', index=False)

In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_128_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_128_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

In [90]:
# Apply k-NN with k=27
knn_classifier = KNeighborsClassifier(n_neighbors=27)
knn_classifier.fit(X_train, y_train)

# Make predictions
y_pred = knn_classifier.predict(X_test)

In [91]:
# Evaluate performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [92]:
# Display the results
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.9769
F1 Score: 0.9769
Precision: 0.9770
Recall: 0.9769
Confusion Matrix:
[[ 971    1    0    0    0    1    4    1    1    1]
 [   0 1128    3    0    0    0    3    0    1    0]
 [   6    0 1000    0    1    0    4   10   10    1]
 [   0    0    3  979    0    8    0   11    7    2]
 [   0    1    0    0  954    0    9    2    1   15]
 [   3    0    0    3    1  869    9    1    3    3]
 [   5    4    0    0    2    3  944    0    0    0]
 [   1   14    9    0    0    0    0  996    0    8]
 [   1    0    2    3    3    4    3    3  950    5]
 [   2    4    0    5    4    2    1    8    5  978]]


# SVM Linear Kernel Combined 128 Feature extracted from RNN and LSTM 

In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_128_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_128_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

In [94]:
# Apply SVM with linear kernel
svm_linear_classifier = SVC(kernel='linear')
svm_linear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_linear = svm_linear_classifier.predict(X_test)

# Evaluate performance metrics for linear kernel
accuracy_linear = accuracy_score(y_test, y_pred_linear)
f1_linear = f1_score(y_test, y_pred_linear, average='weighted')
precision_linear = precision_score(y_test, y_pred_linear, average='weighted')
recall_linear = recall_score(y_test, y_pred_linear, average='weighted')
conf_matrix_linear = confusion_matrix(y_test, y_pred_linear)

In [95]:
# Display the results for linear kernel
print('Linear Kernel Results:')
print(f'Accuracy: {accuracy_linear:.4f}')
print(f'F1 Score: {f1_linear:.4f}')
print(f'Precision: {precision_linear:.4f}')
print(f'Recall: {recall_linear:.4f}')
print(f'ROC AUC: {roc_auc_linear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_linear)

Linear Kernel Results:
Accuracy: 0.9832
F1 Score: 0.9832
Precision: 0.9832
Recall: 0.9832
ROC AUC: 0.9899
Confusion Matrix:
[[ 973    1    0    0    0    0    3    1    2    0]
 [   0 1124    2    2    0    1    2    1    3    0]
 [   1    0 1014    3    1    0    1    8    4    0]
 [   0    0    3  989    0    5    0    6    2    5]
 [   1    2    2    1  965    0    2    1    1    7]
 [   1    0    0    7    1  874    4    0    4    1]
 [   4    2    0    1    3    2  945    0    1    0]
 [   0    1    8    1    1    1    0 1007    1    8]
 [   2    1    3    2    1    2    2    2  956    3]
 [   1    2    0    5    7    3    0    5    1  985]]


# Apply SVM with non-linear kernel (rbf)

In [96]:
svm_nonlinear_classifier = SVC(kernel='rbf') 
svm_nonlinear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_nonlinear = svm_nonlinear_classifier.predict(X_test)

# Evaluate performance metrics for non-linear kernel
accuracy_nonlinear = accuracy_score(y_test, y_pred_nonlinear)
f1_nonlinear = f1_score(y_test, y_pred_nonlinear, average='weighted')
precision_nonlinear = precision_score(y_test, y_pred_nonlinear, average='weighted')
recall_nonlinear = recall_score(y_test, y_pred_nonlinear, average='weighted')
conf_matrix_nonlinear = confusion_matrix(y_test, y_pred_nonlinear)


In [97]:
# Display the results for non-linear kernel
print('Non-Linear Kernel Results:')
print(f'Accuracy: {accuracy_nonlinear:.4f}')
print(f'F1 Score: {f1_nonlinear:.4f}')
print(f'Precision: {precision_nonlinear:.4f}')
print(f'Recall: {recall_nonlinear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_nonlinear)

Non-Linear Kernel Results:
Accuracy: 0.9850
F1 Score: 0.9850
Precision: 0.9850
Recall: 0.9850
Confusion Matrix:
[[ 973    0    2    0    0    1    2    1    1    0]
 [   0 1127    3    1    0    1    1    0    2    0]
 [   3    0 1016    1    1    0    2    5    3    1]
 [   0    0    1  997    0    3    0    3    3    3]
 [   1    0    2    1  965    0    3    1    1    8]
 [   2    0    0    6    1  877    3    0    2    1]
 [   5    3    0    1    1    2  946    0    0    0]
 [   0    4   11    1    1    0    0 1004    2    5]
 [   0    0    3    3    2    2    2    2  956    4]
 [   1    2    0    2    5    3    1    4    2  989]]


# 256 Combined Feature extracted from RNN and LSTM and Train KNN

In [98]:
import pandas as pd
from sklearn.utils import shuffle

# Load the extracted features and labels from both models
lstm_train_df = pd.read_excel('lstm_256_train_features_labels.xlsx')
lstm_test_df = pd.read_excel('lstm_256_test_features_labels.xlsx')
rnn_train_df = pd.read_excel('rnn_256_train_features_labels.xlsx')
rnn_test_df = pd.read_excel('rnn_256_test_features_labels.xlsx')

# Combine features column-wise
combined_train_df = pd.concat([lstm_train_df.drop('label', axis=1), rnn_train_df.drop('label', axis=1), lstm_train_df['label']], axis=1)
combined_test_df = pd.concat([lstm_test_df.drop('label', axis=1), rnn_test_df.drop('label', axis=1), lstm_test_df['label']], axis=1)

# Shuffle the combined data
combined_train_df = shuffle(combined_train_df)
combined_test_df = shuffle(combined_test_df)

# Save the combined features and labels to Excel files
combined_train_df.to_excel('combined_256_train_features_labels.xlsx', index=False)
combined_test_df.to_excel('combined_256_test_features_labels.xlsx', index=False)

In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_256_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_256_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

# Apply k-NN with k=27

In [100]:

knn_classifier = KNeighborsClassifier(n_neighbors=27)
knn_classifier.fit(X_train, y_train)

# Make predictions
y_pred = knn_classifier.predict(X_test)

In [101]:
# Evaluate performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [102]:
# Display the results
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.9768
F1 Score: 0.9768
Precision: 0.9769
Recall: 0.9768
Confusion Matrix:
[[ 974    1    0    0    0    1    1    1    1    1]
 [   0 1130    2    0    0    0    2    0    1    0]
 [   6    2 1000    2    1    0    5    9    7    0]
 [   0    0    2  989    0    1    0    9    5    4]
 [   2    2    0    0  952    0    6    0    2   18]
 [   3    1    0    8    1  862    9    2    1    5]
 [   4    5    0    0    3    1  944    0    1    0]
 [   2   14    6    0    1    0    0  994    1   10]
 [   6    2    2    1    5    5    3    3  942    5]
 [   3    4    0    6    2    1    1    7    4  981]]


# SVM Linear Kernel Combined 256 Feature extracted from RNN and LSTM

In [103]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_256_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_256_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

In [104]:
# Apply SVM with linear kernel
svm_linear_classifier = SVC(kernel='linear')
svm_linear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_linear = svm_linear_classifier.predict(X_test)

# Evaluate performance metrics for linear kernel
accuracy_linear = accuracy_score(y_test, y_pred_linear)
f1_linear = f1_score(y_test, y_pred_linear, average='weighted')
precision_linear = precision_score(y_test, y_pred_linear, average='weighted')
recall_linear = recall_score(y_test, y_pred_linear, average='weighted')
conf_matrix_linear = confusion_matrix(y_test, y_pred_linear)

In [105]:
# Display the results for linear kernel
print('Linear Kernel Results:')
print(f'Accuracy: {accuracy_linear:.4f}')
print(f'F1 Score: {f1_linear:.4f}')
print(f'Precision: {precision_linear:.4f}')
print(f'Recall: {recall_linear:.4f}')
print(f'ROC AUC: {roc_auc_linear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_linear)

Linear Kernel Results:
Accuracy: 0.9845
F1 Score: 0.9845
Precision: 0.9845
Recall: 0.9845
ROC AUC: 0.9899
Confusion Matrix:
[[ 973    1    0    1    1    0    1    1    2    0]
 [   0 1129    2    0    0    1    0    1    2    0]
 [   3    0 1015    4    1    0    1    4    4    0]
 [   0    0    2  997    0    3    0    2    3    3]
 [   0    0    4    0  966    0    4    1    1    6]
 [   2    0    0    5    1  875    2    1    3    3]
 [   3    2    1    1    3    6  941    0    1    0]
 [   1    2    7    0    0    0    0 1009    3    6]
 [   2    1    2    5    2    2    1    2  954    3]
 [   2    2    0    3    8    4    0    2    2  986]]


# Apply SVM with non-linear kernel (rbf)

In [106]:
svm_nonlinear_classifier = SVC(kernel='rbf') 
svm_nonlinear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_nonlinear = svm_nonlinear_classifier.predict(X_test)

# Evaluate performance metrics for non-linear kernel
accuracy_nonlinear = accuracy_score(y_test, y_pred_nonlinear)
f1_nonlinear = f1_score(y_test, y_pred_nonlinear, average='weighted')
precision_nonlinear = precision_score(y_test, y_pred_nonlinear, average='weighted')
recall_nonlinear = recall_score(y_test, y_pred_nonlinear, average='weighted')
conf_matrix_nonlinear = confusion_matrix(y_test, y_pred_nonlinear)


In [107]:
# Display the results for non-linear kernel
print('Non-Linear Kernel Results:')
print(f'Accuracy: {accuracy_nonlinear:.4f}')
print(f'F1 Score: {f1_nonlinear:.4f}')
print(f'Precision: {precision_nonlinear:.4f}')
print(f'Recall: {recall_nonlinear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_nonlinear)

Non-Linear Kernel Results:
Accuracy: 0.9870
F1 Score: 0.9870
Precision: 0.9870
Recall: 0.9870
Confusion Matrix:
[[ 973    1    0    1    0    1    2    1    1    0]
 [   0 1131    2    0    0    1    0    0    1    0]
 [   3    1 1015    2    1    0    1    4    4    1]
 [   0    0    2  998    0    1    0    4    3    2]
 [   1    0    3    0  968    0    3    0    1    6]
 [   2    0    0    5    1  879    2    0    1    2]
 [   2    2    0    1    2    2  947    0    2    0]
 [   1    4    6    0    0    0    0 1010    2    5]
 [   1    0    2    3    1    1    1    3  959    3]
 [   3    2    0    4    6    2    0    2    0  990]]


# Combined 512 Feature extracted from RNN and LSTM and Train KNN

In [1]:
import pandas as pd
from sklearn.utils import shuffle

# Load the extracted features and labels from both models
lstm_train_df = pd.read_excel('lstm_512_train_features_labels.xlsx')
lstm_test_df = pd.read_excel('lstm_512_test_features_labels.xlsx')
rnn_train_df = pd.read_excel('rnn_512_train_features_labels.xlsx')
rnn_test_df = pd.read_excel('rnn_512_test_features_labels.xlsx')

# Combine features column-wise
combined_train_df = pd.concat([lstm_train_df.drop('label', axis=1), rnn_train_df.drop('label', axis=1), lstm_train_df['label']], axis=1)
combined_test_df = pd.concat([lstm_test_df.drop('label', axis=1), rnn_test_df.drop('label', axis=1), lstm_test_df['label']], axis=1)

# Shuffle the combined data
combined_train_df = shuffle(combined_train_df)
combined_test_df = shuffle(combined_test_df)

# Save the combined features and labels to Excel files
combined_train_df.to_excel('combined_512_train_features_labels.xlsx', index=False)
combined_test_df.to_excel('combined_512_test_features_labels.xlsx', index=False)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_512_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_512_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

# Apply k-NN with k=27

In [3]:
knn_classifier = KNeighborsClassifier(n_neighbors=27)
knn_classifier.fit(X_train, y_train)

# Make predictions
y_pred = knn_classifier.predict(X_test)

In [4]:
# Evaluate performance metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [5]:
# Display the results
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.9783
F1 Score: 0.9783
Precision: 0.9784
Recall: 0.9783
Confusion Matrix:
[[ 971    1    0    0    0    1    3    1    2    1]
 [   0 1128    2    0    0    0    3    0    2    0]
 [   6    0 1002    0    2    0    6    7    9    0]
 [   0    0    3  986    0    6    0    6    7    2]
 [   2    2    0    0  950    0    6    0    1   21]
 [   2    0    0    3    1  872    7    1    4    2]
 [   4    2    0    0    3    2  946    0    1    0]
 [   2   11    7    0    2    0    0  994    1   11]
 [   3    0    1    2    4    3    1    3  953    4]
 [   2    3    1    6    4    2    2    5    3  981]]


# SVM Linear Kernel Combined 512 Feature extracted from RNN and LSTM

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt

# Load combined features and labels
combined_train_df = pd.read_excel('combined_512_train_features_labels.xlsx')
combined_test_df = pd.read_excel('combined_512_test_features_labels.xlsx')

# Separate features and labels
X_train = combined_train_df.drop('label', axis=1)
y_train = combined_train_df['label']
X_test = combined_test_df.drop('label', axis=1)
y_test = combined_test_df['label']

In [8]:
# Apply SVM with linear kernel
svm_linear_classifier = SVC(kernel='linear')
svm_linear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_linear = svm_linear_classifier.predict(X_test)

# Evaluate performance metrics for linear kernel
accuracy_linear = accuracy_score(y_test, y_pred_linear)
f1_linear = f1_score(y_test, y_pred_linear, average='weighted')
precision_linear = precision_score(y_test, y_pred_linear, average='weighted')
recall_linear = recall_score(y_test, y_pred_linear, average='weighted')
conf_matrix_linear = confusion_matrix(y_test, y_pred_linear)

In [10]:
# Display the results for linear kernel
print('Linear Kernel Results:')
print(f'Accuracy: {accuracy_linear:.4f}')
print(f'F1 Score: {f1_linear:.4f}')
print(f'Precision: {precision_linear:.4f}')
print(f'Recall: {recall_linear:.4f}')

print('Confusion Matrix:')
print(conf_matrix_linear)

Linear Kernel Results:
Accuracy: 0.9859
F1 Score: 0.9859
Precision: 0.9859
Recall: 0.9859
Confusion Matrix:
[[ 971    1    1    0    1    0    3    1    2    0]
 [   0 1129    1    1    0    0    1    1    2    0]
 [   1    0 1018    2    1    0    1    3    5    1]
 [   0    0    2  998    0    4    0    3    2    1]
 [   1    1    3    0  964    0    3    1    1    8]
 [   2    0    0    6    1  877    2    0    2    2]
 [   3    2    0    1    2    4  946    0    0    0]
 [   2    2    7    1    1    0    0 1008    2    5]
 [   2    0    2    3    3    2    0    2  955    5]
 [   1    2    0    1    7    2    0    3    0  993]]


# Apply SVM with non-linear kernel (rbf) (512 features) 

In [11]:
svm_nonlinear_classifier = SVC(kernel='rbf') 
svm_nonlinear_classifier.fit(X_train, y_train)

# Make predictions
y_pred_nonlinear = svm_nonlinear_classifier.predict(X_test)

# Evaluate performance metrics for non-linear kernel
accuracy_nonlinear = accuracy_score(y_test, y_pred_nonlinear)
f1_nonlinear = f1_score(y_test, y_pred_nonlinear, average='weighted')
precision_nonlinear = precision_score(y_test, y_pred_nonlinear, average='weighted')
recall_nonlinear = recall_score(y_test, y_pred_nonlinear, average='weighted')
conf_matrix_nonlinear = confusion_matrix(y_test, y_pred_nonlinear)


In [12]:
# Display the results for non-linear kernel
print('Non-Linear Kernel Results:')
print(f'Accuracy: {accuracy_nonlinear:.4f}')
print(f'F1 Score: {f1_nonlinear:.4f}')
print(f'Precision: {precision_nonlinear:.4f}')
print(f'Recall: {recall_nonlinear:.4f}')
print('Confusion Matrix:')
print(conf_matrix_nonlinear)

Non-Linear Kernel Results:
Accuracy: 0.9867
F1 Score: 0.9867
Precision: 0.9867
Recall: 0.9867
Confusion Matrix:
[[ 973    1    0    0    0    0    2    2    2    0]
 [   0 1129    1    2    0    1    1    0    1    0]
 [   1    1 1017    2    1    0    2    3    4    1]
 [   0    0    2  997    0    4    0    3    2    2]
 [   1    1    3    0  964    0    3    1    1    8]
 [   2    0    0    6    1  878    2    0    2    1]
 [   3    2    0    1    1    3  946    0    2    0]
 [   1    3    6    1    1    0    0 1010    2    4]
 [   1    0    2    3    2    2    0    2  958    4]
 [   0    2    0    1    6    2    1    2    0  995]]
